HuuggingFace

In [1]:
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [ ]:
sys.path

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
import torch
print(torch.cuda.is_available())

True


In [1]:
from transformers import __version__
print(f'Transformers version: {__version__}')
from torch import __version__

print(f'torch version: {__version__}')

Transformers version: 4.32.1
torch version: 2.2.1+cu121


pip install torch

In [3]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]


KeyboardInterrupt



pip install torch -U

pip install transformers -U

pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

pip install accelerate

pip install bitsandbytes

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
# from accelerate import Accelerator


In [5]:
torch.cuda.empty_cache()

In [6]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [8]:
# mn = "meta-llama/Llama-2-7b-chat-hf"
# model = AutoModelForCausalLM.from_pretrained(mn)

C:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(

KeyboardInterrupt



In [8]:
mn = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(mn, device_map="auto")#, load_in_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "What is dog?"
toks = tokr(prompt, return_tensors="pt")
toks

{'input_ids': tensor([[    1,  1724,   338, 11203, 29973]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [10]:
tokr.batch_decode(toks['input_ids'])

['<s> What is dog?']

In [14]:
res = model.generate(**toks.to("cuda"), max_new_tokens=30)
res

tensor([[    1,  1724,   338, 11203, 29973,    13, 29928, 12099,   313,  6028,
           275,   301,   786,   375,  9985,   275, 29897,   526, 21849,   630,
           286,  4850,  1338,   322,   263,  5972,  5697, 29889,  2688,   526,
         16467,  4475,   304, 20040,  1960]], device='cuda:0')

In [17]:
text = tokr.batch_decode(res)

In [18]:
print(text[0])

<s> What is dog?
Dogs (Canis lupus familiaris) are domesticated mammals and a popular pet. They are closely related to wolves


In [32]:
def gen(p, maxlen=15, sample=True):
    if not sample:
        temp = None
    else:
        temp = 0.6
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample, temperature=temp)
    return tokr.batch_decode(res)

In [33]:
ask = "what is Foxconn?"
ans = gen(p=ask, maxlen=20, sample=False)
ans

C:\ProgramData\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['<s> what is Foxconn?\n\nFoxconn is a multinational contract manufacturing company based in Taiwan. It is']

In [35]:
ans[0]

'<s> what is Foxconn?\n\nFoxconn is a multinational contract manufacturing company based in Taiwan. It is'